In [45]:
#workaround for running jupyter from wsl
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))

In [46]:
from dotenv import load_dotenv

load_dotenv()

True

In [47]:
uri = "http://ramp.uni-mannheim.de"
graph_endpoint = os.getenv("GRAPH_ENDPOINT")
username = os.getenv("GRAPH_USERNAME")
password = os.getenv("GRAPH_PASSWORD")

In [48]:
# Initialize an RDFLib Graph
from rdflib import Graph, Namespace

RAMP = Namespace("http://ramp.uni-mannheim.de/")

graph = Graph()

graph.bind("ramp", RAMP)

print(os.getcwd())

# Load ontology from the specified file
graph.parse("./ontology/ontology.ttl", format="turtle")

/Users/yannikhahn/Code/module-catalog-extractor/sparql


<Graph identifier=N565497aa470f478d8955026894f14101 (<class 'rdflib.graph.Graph'>)>

In [49]:
from scripts import parse_module_catalog, parse_overview


overview = parse_overview("../output/20241112_000328/catalog_overview.json")

split = parse_module_catalog("../output/20241112_000328/split_2_modules_list.json")

In [50]:
from rdflib import URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS
import requests
import hashlib
m = hashlib.md5()

XSD = Namespace("http://www.w3.org/2001/XMLSchema#")
FOAF = Namespace("http://xmlns.com/foaf/0.1/")

# Iterate through each study area and module to create triples
for study_area in overview.studyArea:
    m.update(study_area.name.encode())
    # Create the URI for the study area
    study_area_uri = URIRef(f"{uri}/study_area/{str(int(m.hexdigest(), 16))[0:12]}")
    graph.add((study_area_uri, RDF.type, RAMP.StudyArea))
    graph.add((study_area_uri, RDFS.label, Literal(study_area.name)))
    for ects in study_area.requiredEcts:
        graph.add((study_area_uri, RAMP.ects, Literal(ects, datatype=XSD.integer)))
    
    for module in study_area.modules:
        # Create the URI for the module
        module_uri = URIRef(f"{uri}/module/{module.id}")

        # Add RDF type triple
        graph.add((module_uri, RDF.type, RAMP.Module))

        # Add relationship between study area and module
        graph.add((study_area_uri, RAMP.hasModule, module_uri))
        #graph.add((module_uri, RAMP.isPartOfStudyArea, study_area_uri))
        
        # Add literal properties for name and id
        graph.add((module_uri, RDFS.label, Literal(module.name)))
        graph.add((module_uri, RAMP.id, Literal(module.id)))

rdf_data = graph.serialize(format="nt")

# graph.update(f"INSERT DATA {{ {graph.serialize(format='nt')} }}")



In [51]:
from rdflib import BNode


for module in split.modules:
    module_uri = URIRef(f"{uri}/module/{module.id}")

    if module.ects is not None:
        graph.add((module_uri, RAMP.ects, Literal(module.ects, datatype=XSD.int)))
    if module.workloadInPerson is not None:
        graph.add((module_uri, RAMP.workloadInPerson, Literal(module.workloadInPerson, datatype=XSD.int)))
    if module.workloadSelfStudy is not None:
        graph.add((module_uri, RAMP.workloadSelfStudy, Literal(module.workloadSelfStudy, datatype=XSD.int)))
    if module.assessmentForms:
        for i, form in enumerate(module.assessmentForms):
            blank_node = BNode()
            graph.add((module_uri, RAMP.hasAssesment, blank_node))
            graph.add((blank_node, RAMP.hasAssessmentForm, Literal(form.value)))
            if module.examinationDistribution:
                if(len(module.examinationDistribution.split("/")) > i):
                    dist = module.examinationDistribution.split("/")[i] 
                    graph.add((blank_node, RAMP.examinationDistribution, Literal(dist, datatype=XSD.int)))

    if module.examinationDuration is not None:
        graph.add((module_uri, RAMP.examinationDuration, Literal(module.examinationDuration, datatype=XSD.int)))
    if module.additionalPrerequisite:
        graph.add((module_uri, RAMP.additionalPrerequisite, Literal(module.additionalPrerequisite)))
    # Add lists of literals
    for semester in module.offeredIn:
        graph.add((module_uri, RAMP.offeredIn, Literal(semester.value)))
    for recommendedSemester in module.recommendedSemester:
        graph.add((module_uri, RAMP.recommendedSemester, Literal(recommendedSemester, datatype=XSD.int)))
    for literature in module.recommendedLiterature:
        graph.add((module_uri, RAMP.recommendedLiterature, Literal(literature)))
    for prerequisite in module.prerequisiteModules:
        graph.add((module_uri, RAMP.hasPrerequisite, URIRef(f"{uri}/module/{prerequisite}")))
    for further in module.furtherModules:
        graph.add((module_uri, RAMP.hasFurtherModue, Literal(further)))

     # Add lecturer and person in charge as URI references
    for lecturer in module.hasLecturer:
        if lecturer.isPerson:
            lecturer_uri = URIRef(f"{uri}/person/{lecturer.name.replace(' ', '_')}")
            graph.add((module_uri, RAMP.hasLecturer, lecturer_uri))
        graph.add((lecturer_uri, RDF.type, FOAF.Person))
    for person_in_charge in module.hasPersonInCharge:
        if person_in_charge.isPerson:
            person_in_charge_uri = URIRef(f"{uri}/person/{person_in_charge.name.replace(' ', '_')}")
            graph.add((module_uri, RAMP.hasPersonInCharge, person_in_charge_uri))
            graph.add((person_in_charge_uri, RDF.type, FOAF.Person))
    


In [52]:
# Serialize and print the graph in Turtle format (optional for debugging)
print(graph.serialize(format="turtle"))

# store ontology in file
graph.serialize(destination="../ontology/ontology-full.ttl", format="turtle")

rdf_data = graph.serialize(format="turtle")

@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix ramp: <http://ramp.uni-mannheim.de/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://ramp.uni-mannheim.de> a owl:Ontology .

ramp:additionalPrerequisite a owl:DatatypeProperty ;
    rdfs:domain ramp:Module .

ramp:ects a owl:DatatypeProperty ;
    rdfs:label "ECTS" ;
    rdfs:domain ramp:Module,
        ramp:StudyArea ;
    rdfs:range xsd:decimal .

ramp:examinationDistribution a owl:AnnotationProperty ;
    rdfs:domain ramp:Module ;
    rdfs:range ramp:AssessmentForm .

ramp:examinationDuration a owl:AnnotationProperty ;
    rdfs:domain ramp:Module ;
    rdfs:range ramp:AssessmentForm .

ramp:hasDegree a owl:ObjectProperty ;
    rdfs:domain ramp:StudyProgram ;
    rdfs:range ramp:Degree .

ramp:hasExaminationForm a owl:ObjectProperty .

ramp:hasLecturer a o

In [53]:
from blazegraph_client import create_blazegraph_namespace, delete_blazegraph_namespace

#reset graph

delete_blazegraph_namespace("ramp")

create_blazegraph_namespace("ramp")




Namespace 'ramp' deleted successfully.
Namespace 'ramp' created successfully.


True

In [54]:


from owlready2 import get_ontology, sync_reasoner
import io
rdf_data_xml = graph.serialize(format="xml")
rdf_file = io.BytesIO(rdf_data_xml.encode("utf-8"))

# Load serialized RDF/XML data as an ontology in Owlready2
# (using a temporary URI since the data is in-memory)
onto = get_ontology("http://ramp.uni-mannheim.de").load(fileobj=rdf_file)
with onto:
    # Run Owlready2's reasoner
    sync_reasoner()

reasoned_graph = onto.world.as_rdflib_graph()
reasoned_graph.bind("ramp", RAMP)

reasoned_turtle = reasoned_graph.serialize(format="turtle")
reasoned_graph.serialize(destination="../ontology/ontology-reasoned.ttl", format="turtle")


* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /Users/yannikhahn/Library/Caches/pypoetry/virtualenvs/module-catalog-extractor-49STR4rT-py3.12/lib/python3.12/site-packages/owlready2/hermit:/Users/yannikhahn/Library/Caches/pypoetry/virtualenvs/module-catalog-extractor-49STR4rT-py3.12/lib/python3.12/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////var/folders/zx/63q9k_gd4nb5sbdwq12wb_km0000gn/T/tmp9yaqsthq
* Owlready2 * HermiT took 0.7304551601409912 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


<Graph identifier=Ne95d08234d444b34a65b7c4f8e55f2a7 (<class 'owlready2.rdflib_store.TripleLiteRDFlibGraph'>)>

In [55]:
from requests.auth import HTTPBasicAuth
# Send the update request to the SPARQL endpoint
# Prepare SPARQL INSERT statement
sparql_update = f"""
INSERT DATA {{
    {rdf_data}
}}
"""

# Send the request with Basic Authentication
headers = {
    "Content-Type": "application/sparql-update"
}

response = requests.post(
    url=graph_endpoint,
    data=sparql_update,
    headers=headers,
    auth=HTTPBasicAuth(username, password)
)

if response.status_code == 200:
        print("Data successfully inserted.")
else:
    print("Failed to insert data:", response.status_code, response.text)

Data successfully inserted.
